In [ ]:
# =========================================================
# SEÇÃO 1: IMPORTAÇÕES E SETUP GERAL
# =========================================================

In [ ]:
import os
import copy
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
from functools import partial

# --- Nossas funções auxiliares agora são importadas! ---
from helper.utils import (
    definir_seed,
    carregar_serie,
    dividir_serie_temporal,
    preparar_dados_para_neuralforecast
)

# --- Importações específicas para o treinamento ---
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, MLP, LSTM, Autoformer, NHITS

warnings.filterwarnings("ignore")

In [ ]:
# =========================================================
# SEÇÃO 2: PIPELINE DE EXPERIMENTO
# =========================================================

In [ ]:
def encontrar_melhor_arima_auto(treino_log, freq):
    # (Mantenha sua função original aqui)
    print("Buscando melhor ordem ARIMA com auto_arima...")
    m = 12 if freq.startswith('M') else (4 if freq.startswith('Q') else 1)
    auto_arima_model = pm.auto_arima(treino_log, m=m, seasonal=True, trace=False,
                                     error_action='ignore', suppress_warnings=True, stepwise=True)
    print(
        f"Melhor ordem encontrada: {auto_arima_model.order} Sazonal: {auto_arima_model.seasonal_order}")
    return auto_arima_model.order, auto_arima_model.seasonal_order

In [ ]:
import copy

def executar_experimento(nome_da_serie, horizonte):
    """
    Versão final e COMPLETA do pipeline, incluindo:
    1. Modelo ARIMA.
    2. Modelos Neurais Puros (não-híbridos).
    3. Múltiplos Sistemas Híbridos (MIMO e Recursive).
    """
    try:
        # --- Configurações Iniciais ---
        SEED = 42
        definir_seed(SEED)
        INPUT_SIZE_NEURAL = 50 
        MAX_STEPS_NEURAL = 200 
        
        # --- Carga e Preparação ---
        serie_original = carregar_serie(nome_da_serie)
        percentual_treino = 1 - (horizonte / len(serie_original))
        
        tamanho_treino_necessario = INPUT_SIZE_NEURAL + horizonte
        if len(serie_original) * percentual_treino < tamanho_treino_necessario:
            print(f"AVISO: A série '{nome_da_serie}' (treino de {int(len(serie_original) * percentual_treino)} pts) é muito curta para o input_size={INPUT_SIZE_NEURAL} e horizonte={horizonte}. Pulando.")
            return None

        treino_orig, teste_orig = dividir_serie_temporal(serie_original, percentual_treino=percentual_treino)
        serie_log = np.log(serie_original)
        treino_log, _ = dividir_serie_temporal(serie_log, percentual_treino=percentual_treino)
        freq = serie_original.index.freqstr or pd.infer_freq(serie_original.index)
        if freq is None: return None
        previsoes_teste = {'y_true': teste_orig.values}
        
        # --- 1. Modelo ARIMA ---
        modelo_arima = None
        try:
            print(f"Processando: ARIMA para '{nome_da_serie}'")
            ordem, ordem_sazonal = encontrar_melhor_arima_auto(treino_log, freq)
            modelo_arima = ARIMA(treino_log.asfreq(freq), order=ordem, seasonal_order=ordem_sazonal).fit()
            preds_log_teste_arima = modelo_arima.forecast(steps=horizonte)
            previsoes_teste['ARIMA'] = np.exp(preds_log_teste_arima).values
        except Exception as e: 
            print(f"AVISO: ARIMA falhou para '{nome_da_serie}': {e}")
            return None

        # ==============================================================================
        # --- 2. Modelos Neurais Puros (Não-Híbridos) ---
        # ==============================================================================
        print("\n--- Processando Modelos Neurais Puros ---")
        df_treino_log_nf = preparar_dados_para_neuralforecast(treino_log, nome_da_serie)
        modelos_para_testar = {
            'N-BEATS': NBEATS, 
            'MLP': MLP, 
            'LSTM': LSTM, 
            'Autoformer': Autoformer, 
            'NHITS': NHITS
        }
        
        for nome_modelo, classe_modelo in modelos_para_testar.items():
            try:
                print(f"Processando: {nome_modelo} (Puro)")
                modelo_neural = [classe_modelo(input_size=INPUT_SIZE_NEURAL, h=horizonte, max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
                nf = NeuralForecast(models=modelo_neural, freq=freq)
                nf.fit(df=df_treino_log_nf, verbose=False)
                previsoes_teste[nome_modelo] = np.exp(nf.predict()[classe_modelo.__name__].values)
            except Exception as e: 
                print(f"AVISO: {nome_modelo} (Puro) falhou: {e}")
                previsoes_teste[nome_modelo] = np.nan
        
        # --- Preparação dos Resíduos para Modelos Híbridos ---
        print("\n--- Processando Modelos Híbridos ---")
        residuos_treino_log = modelo_arima.resid
        df_residuos_treino = preparar_dados_para_neuralforecast(residuos_treino_log, "residuos")
        
        # --- 3. HÍBRIDO: ARIMA + N-BEATS (MIMO) ---
        try:
            print("Processando: Híbrido N-BEATS (MIMO)")
            modelo_nbeats_mimo = [NBEATS(input_size=INPUT_SIZE_NEURAL, h=horizonte, max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
            nf_nbeats_mimo = NeuralForecast(models=modelo_nbeats_mimo, freq=freq)
            nf_nbeats_mimo.fit(df=df_residuos_treino, verbose=False)
            preds_residuos_mimo = nf_nbeats_mimo.predict()['NBEATS'].values
            previsoes_teste['Híbrido N-BEATS (MIMO)'] = previsoes_teste['ARIMA'] + preds_residuos_mimo
        except Exception as e:
            previsoes_teste['Híbrido N-BEATS (MIMO)'] = np.nan

        # --- 4. HÍBRIDO: ARIMA + MLP (MIMO) ---
        try:
            print("Processando: Híbrido MLP (MIMO)")
            modelo_mlp_mimo = [MLP(input_size=INPUT_SIZE_NEURAL, h=horizonte, max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
            nf_mlp_mimo = NeuralForecast(models=modelo_mlp_mimo, freq=freq)
            nf_mlp_mimo.fit(df=df_residuos_treino, verbose=False)
            preds_residuos_mlp_mimo = nf_mlp_mimo.predict()['MLP'].values
            previsoes_teste['Híbrido MLP (MIMO)'] = previsoes_teste['ARIMA'] + preds_residuos_mlp_mimo
        except Exception as e:
            previsoes_teste['Híbrido MLP (MIMO)'] = np.nan

        # --- 5. HÍBRIDO: ARIMA + LSTM (MIMO) ---
        try:
            print("Processando: Híbrido LSTM (MIMO)")
            modelo_lstm_mimo = [LSTM(input_size=INPUT_SIZE_NEURAL, h=horizonte, max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
            nf_lstm_mimo = NeuralForecast(models=modelo_lstm_mimo, freq=freq)
            nf_lstm_mimo.fit(df=df_residuos_treino, verbose=False)
            preds_residuos_lstm_mimo = nf_lstm_mimo.predict()['LSTM'].values
            previsoes_teste['Híbrido LSTM (MIMO)'] = previsoes_teste['ARIMA'] + preds_residuos_lstm_mimo
        except Exception as e:
            previsoes_teste['Híbrido LSTM (MIMO)'] = np.nan

        # --- 6. HÍBRIDO: ARIMA + N-BEATS (Recursive-Direct) ---
        try:
            print("Processando: Híbrido N-BEATS (Recursive-Direct)")
            
            # 1. Preparação dos dados em formato Painel
            lista_dfs_horizontes = []
            for h_step in range(1, horizonte + 1):
                df_residuos_h = residuos_treino_log.to_frame(name='y')
                df_residuos_h['ds'] = df_residuos_h.index
                df_residuos_h['y'] = df_residuos_h['y'].shift(-h_step + 1)
                df_residuos_h['unique_id'] = f'h_{h_step}'
                df_residuos_h.dropna(inplace=True)
                
                if len(df_residuos_h) >= INPUT_SIZE_NEURAL + 1:
                    lista_dfs_horizontes.append(df_residuos_h)

            if not lista_dfs_horizontes:
                 raise ValueError("Nenhum horizonte tinha dados suficientes para treinar o modelo 'Direct'.")

            df_treino_residuos_painel = pd.concat(lista_dfs_horizontes)

            # 2. Treinamento e Previsão
            modelo_direct = [NBEATS(input_size=INPUT_SIZE_NEURAL, h=1, max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
            nf_direct = NeuralForecast(models=modelo_direct, freq=freq)
            nf_direct.fit(df=df_treino_residuos_painel, verbose=False)
            preds_df = nf_direct.predict()
            
            # 3. Organização e Combinação dos Resultados
            preds_df['h_step'] = preds_df['unique_id'].str.replace('h_', '').astype(int)
            preds_df.sort_values(by='h_step', inplace=True)
            preds_residuos_direct = preds_df['NBEATS'].values

            if len(preds_residuos_direct) < horizonte:
                preds_completas = np.zeros(horizonte)
                indices_validos = preds_df['h_step'].values - 1
                preds_completas[indices_validos] = preds_residuos_direct
                preds_residuos_direct = preds_completas

            previsoes_teste['Híbrido N-BEATS (Direct)'] = previsoes_teste['ARIMA'] + preds_residuos_direct

        except Exception as e: 
            print(f"AVISO: Híbrido N-BEATS (Direct) falhou: {e}")
            previsoes_teste['Híbrido N-BEATS (Direct)'] = np.nan
            
        # --- 7. HÍBRIDO: ARIMA + N-BEATS (Recursive) ---
        try:
            print("Processando: Híbrido N-BEATS (Recursive)")
            modelo_nbeats_rec = [NBEATS(input_size=INPUT_SIZE_NEURAL, h=1, max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
            nf_nbeats_rec = NeuralForecast(models=modelo_nbeats_rec, freq=freq)
            nf_nbeats_rec.fit(df=df_residuos_treino, verbose=False)
            preds_residuos_rec_df = nf_nbeats_rec.predict(h=horizonte)
            preds_residuos_log_rec = preds_residuos_rec_df['NBEATS'].values
            previsoes_teste['Híbrido N-BEATS (Recursive)'] = previsoes_teste['ARIMA'] + np.array(preds_residuos_log_rec)
        except Exception as e: 
            previsoes_teste['Híbrido N-BEATS (Recursive)'] = np.nan

        # --- Finalização ---
        df_final = pd.DataFrame(previsoes_teste, index=teste_orig.index)
        df_final['dataset'] = nome_da_serie
        df_final['horizonte'] = horizonte
        return df_final.reset_index().rename(columns={'index': 'ds'})
    
    except Exception as e:
        import traceback
        print(f"ERRO GERAL no processamento de '{nome_da_serie}' para o horizonte {horizonte}: {e}")
        traceback.print_exc()
        return None

In [ ]:
# =========================================================
# SEÇÃO 3: ORQUESTRADOR (LÓGICA DA CAMADA SILVER)
# =========================================================

In [ ]:
# --- Configurações ---
LISTA_DE_DATASETS = ['AirPassengers', 'co2', 'nottem', 'JohnsonJohnson', 'UKgas']
# LISTA_DE_DATASETS = ['AirPassengers']
VETOR_DE_HORIZONTES = [10, 12, 15, 24]
output_dir = "./data/silver"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "resultados_completos.csv")

# --- Lógica de Execução Incremental ---
# Apaga o arquivo antigo para garantir uma execução limpa
if os.path.exists(output_file):
    os.remove(output_file)

header_escrito = False

In [ ]:
for dataset in tqdm(LISTA_DE_DATASETS, desc="Processando Datasets"):
    for horizonte in tqdm(VETOR_DE_HORIZONTES, desc=f"Testando Horizontes para {dataset}", leave=False):
        # Executa o experimento para uma combinação de dataset e horizonte
        df_resultado_detalhado = executar_experimento(dataset, horizonte)

        if df_resultado_detalhado is not None:
            # Lógica para salvar incrementalmente no arquivo da camada Silver
            if not header_escrito:
                # Escreve o cabeçalho apenas na primeira vez
                df_resultado_detalhado.to_csv(
                    output_file, index=False, mode='w', header=True)
                header_escrito = True
            else:
                # Anexa os novos resultados sem o cabeçalho
                df_resultado_detalhado.to_csv(
                    output_file, index=False, mode='a', header=False)

print(f"\nProcesso finalizado. Resultados salvos em '{output_file}'")

Epoch 104:   0%|          | 0/1 [00:00<?, ?it/s, v_num=45, train_loss_step=0.0334, train_loss_epoch=0.0334]        


Detected KeyboardInterrupt, attempting graceful shutdown ...
Seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | mlp          | ModuleList    | 1.1 M  | train
4 | out          | Linear        | 10.2 K | train
-------------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.448     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


AVISO: N-BEATS (Puro) falhou: name 'exit' is not defined
Processando: MLP (Puro)
Epoch 86:   0%|          | 0/1 [00:00<?, ?it/s, v_num=46, train_loss_step=0.0636, train_loss_epoch=0.0636]        